# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,labasa,-16.4167,179.3833,26.10,67,100,2.53,FJ,1731098784
1,1,wailua homesteads,22.0669,-159.3780,26.44,71,75,8.75,US,1731098785
2,2,port-aux-francais,-49.3500,70.2167,3.91,65,1,8.09,TF,1731098786
3,3,kingston,17.9970,-76.7936,28.98,89,40,6.17,JM,1731098787
4,4,morant bay,17.8815,-76.4093,28.03,80,31,3.71,JM,1731098788


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = True,
    frame_width = 700,
    frame_height = 500,
    tiles = "OSM"
)

# Display the map
map_plot_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Narrow down cities that fit criteria for ideal weather conditions
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 15) & (city_data_df['Max Temp'] <= 20) &  
    (city_data_df['Humidity'] < 60) &  
    (city_data_df['Cloudiness'] < 50)  
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,hamilton,39.1834,-84.5333,19.13,42,0,2.24,US,1731098814
52,52,bethel,41.3712,-73.4140,18.09,40,0,2.68,US,1731098843
96,96,park city,42.3484,-87.8842,16.03,47,0,3.09,US,1731098889
139,139,crane,31.3974,-102.3501,16.75,30,1,8.09,US,1731098733
223,223,tamanrasset,22.7850,5.5228,16.95,27,0,2.06,DZ,1731099029
249,249,north battleford,52.7834,-108.2847,16.05,39,0,5.14,CA,1731098875
254,254,idri,27.5000,13.2667,19.07,40,0,2.66,LY,1731099063
266,266,pahrump,36.2083,-115.9839,17.36,12,0,0.89,US,1731098918
284,284,palmer,42.1584,-72.3287,16.75,38,40,6.69,US,1731099096
425,425,east tamworth,-31.0894,150.9436,17.77,52,0,4.63,AU,1731099256


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
29,hamilton,US,39.1834,-84.5333,42,
52,bethel,US,41.3712,-73.4140,40,
96,park city,US,42.3484,-87.8842,47,
139,crane,US,31.3974,-102.3501,30,
223,tamanrasset,DZ,22.7850,5.5228,27,
249,north battleford,CA,52.7834,-108.2847,39,
254,idri,LY,27.5000,13.2667,40,
266,pahrump,US,36.2083,-115.9839,12,
284,palmer,US,42.1584,-72.3287,38,
425,east tamworth,AU,-31.0894,150.9436,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
     "categories": "accommodation.hotel",  
    "limit": 1,  
    "apiKey": geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    response = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
bethel - nearest hotel: Hampton Inn Danbury
park city - nearest hotel: Red Roof Inn Gurnee - Waukegan
crane - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
north battleford - nearest hotel: Beaver Hotel
idri - nearest hotel: No hotel found
pahrump - nearest hotel: Holiday Inn Express & Suites
palmer - nearest hotel: No hotel found
east tamworth - nearest hotel: Quality Hotel Powerhouse
sa'dah - nearest hotel: الشارقة مول للمواد الغذائية
paphos - nearest hotel: Mayfair Gardens Hotel
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country


,City,Country,Lat,Lng,Humidity,Hotel Name
29,hamilton,US,39.1834,-84.5333,42,North Vista Manor
52,bethel,US,41.3712,-73.4140,40,Hampton Inn Danbury
96,park city,US,42.3484,-87.8842,47,Red Roof Inn Gurnee - Waukegan
139,crane,US,31.3974,-102.3501,30,No hotel found
223,tamanrasset,DZ,22.7850,5.5228,27,فندق الأمان
249,north battleford,CA,52.7834,-108.2847,39,Beaver Hotel
254,idri,LY,27.5000,13.2667,40,No hotel found
266,pahrump,US,36.2083,-115.9839,12,Holiday Inn Express & Suites
284,palmer,US,42.1584,-72.3287,38,No hotel found
425,east tamworth,AU,-31.0894,150.9436,52,Quality Hotel Powerhouse


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = True,
    frame_width = 700,
    frame_height = 500,
    tiles = "OSM",
    hover_cols=["City", "Country", "Hotel Name"],
     title="Hotel Locations"
)

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)